<a href="https://colab.research.google.com/github/rsin46/pegasus-demo/blob/master/Pegasus_Combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training/converting to .PB

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
%cd /content/

/content


In [10]:
!git clone https://github.com/google-research/pegasus
!pip3 install tensorflow_transform
!pip3 install -e pegasus
%cd pegasus
!export PYTHONPATH=.

Cloning into 'pegasus'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 353.09 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (56/56), done.
     |████████████████████████████████| 368kB 7.0MB/s 
     |████████████████████████████████| 8.3MB 23.0MB/s 
     |████████████████████████████████| 3.0MB 29.1MB/s 
     |████████████████████████████████| 112kB 42.6MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 829kB 37.0MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 153kB 42.4MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 63.8MB 145kB/s 
     |████████████████████████████████| 1.4MB 35.4MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
     |████████████████████████████████| 215kB 3

Obtaining file:///content/pegasus
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 1.1MB 10.1MB/s 
     |████████████████████████████████| 8.6MB 32.3MB/s 
     |████████████████████████████████| 3.5MB 33.4MB/s 
     |████████████████████████████████| 1.4MB 39.4MB/s 
     |████████████████████████████████| 411.0MB 44kB/s 
     |████████████████████████████████| 110.5MB 45kB/s 
     |████████████████████████████████| 368kB 37.5MB/s 
     |████████████████████████████████| 655kB 33.8MB/s 
     |████████████████████████████████| 983kB 17.4MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 358kB 38.8MB/s 
     |████████████████████████████████| 5.3MB 35.2MB/s 
     |████████████████████████████████| 184kB 40.7MB/s 
     |████████████████████████████████| 307kB 45.9MB/s 
     |████████████████████████████████| 512kB 33.6MB/s 
     |████████████████████████████████| 3.8MB 35.1MB/s 
     |███████████

## Training fresh data from checkpoint

### Base model (no finetuning)

In [11]:
%cd /content/pegasus/

/content/pegasus


In [12]:
!mkdir -p ckpt/pegasus_ckpt
!gsutil -m rsync gs://pegasus_ckpt ckpt/pegasus_ckpt/

Building synchronization state...
Starting synchronization...
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
Copying gs://pegasus_ckpt/model.ckpt-1500000.meta...
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab...
Copying gs://pegasus_ckpt/model.ckpt-1500000.data-00000-of-00001...
Copying gs://pegasus_ckpt/model.ckpt-1500000.index...
Copying gs://pegasus_ckpt/checkpoint...
- [6/6 files][  2.3 GiB/  2.3 GiB] 100% Done  41.0 MiB/s ETA 00:00:00           
Operation completed over 6 objects/2.3 GiB.                                      


## Converting trained model to .pb (vary batch size)

In [16]:
%cd /content/pegasus

/content/pegasus


In [17]:
!gsutil cp -r gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model /content/pegasus/ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model

Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


In [18]:
!cp -r '/content/gdrive/My Drive/Models/arxiv/' '/content/pegasus/ckpt/pegasus_ckpt/'

cp: cannot stat '/content/gdrive/My Drive/Models/arxiv/': No such file or directory


In [19]:
MODELNAME = 'xsum'
BATCH_SIZE = 4
max_input_len = 1024
max_target_len = 64

import itertools
import os
import time

from absl import logging
from pegasus.data import infeed
from pegasus.params import all_params  # pylint: disable=unused-import
from pegasus.params import estimator_utils
from pegasus.params import registry
import tensorflow as tf
from pegasus.eval import text_eval
from pegasus.ops import public_parsing_ops
import pandas as pd
from random import choice
from tensorflow.python.estimator.export import export
tf.enable_eager_execution()
# import tensorflow_transform as tft
data_name = MODELNAME
import tensorflow_transform as tft

master = ""
# model_dir = "./ckpt/pegasus_ckpt/%s"%data_name
model_dir = "./ckpt/pegasus_ckpt/"
use_tpu = False
iterations_per_loop = 1000
num_shards = 1
param_overrides = ("vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model"
",batch_size={0},beam_size=5,beam_alpha=0.6,max_input_len={1},max_target_len={2}".format(BATCH_SIZE, max_input_len, max_target_len))

eval_dir = os.path.dirname(model_dir)
checkpoint_path = model_dir
checkpoint_path = tf.train.latest_checkpoint(checkpoint_path )
params = registry.get_params('%s_transformer'%data_name)(param_overrides)
pattern = params.dev_pattern
input_fn = infeed.get_input_fn(params.parser, pattern,
                                    tf.estimator.ModeKeys.PREDICT)
parser, shapes = params.parser(mode=tf.estimator.ModeKeys.PREDICT)
RAW_DATA_FEATURE_SPEC = dict([("inputs", tf.io.FixedLenFeature(shapes['inputs'], tf.int64)), 
                              ('targets', tf.io.FixedLenFeature(shapes['targets'], tf.string))])

raw_feature_spec = RAW_DATA_FEATURE_SPEC.copy()
raw_feature_spec.pop('targets')
# raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
#         raw_feature_spec, default_batch_size=0)
def serving_input_fn():
    raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec, default_batch_size=BATCH_SIZE)
    serving_input_receiver = raw_input_fn()

    raw_features = serving_input_receiver.features
    return tf.estimator.export.ServingInputReceiver(
        raw_features, serving_input_receiver.receiver_tensors)

print(tf.executing_eagerly())
estimator = estimator_utils.create_estimator(master, 
                                            model_dir,
                                            use_tpu,
                                            iterations_per_loop,
                                            num_shards, params, include_features_in_predictions=False)

output = estimator.export_saved_model(
    "model/", serving_input_fn
)


True

INFO:tensorflow:Using config: {'_model_dir': './ckpt/pegasus_ckpt/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fca0436e0b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infe

In [20]:
print(shapes)

{'inputs': [1024], 'targets': [64]}


In [21]:
dir = '/content/pegasus/' + output.decode("utf-8")
!ls -l $dir/variables/

total 2220748
-rw-r--r-- 1 root root 2274013192 Aug 16 07:53 variables.data-00000-of-00001
-rw-r--r-- 1 root root      21694 Aug 16 07:53 variables.index


In [22]:
!cp -rv $dir '/content/gdrive/My Drive/Models/'

'/content/pegasus/model/1597564312' -> '/content/gdrive/My Drive/Models/1597564312'
'/content/pegasus/model/1597564312/variables' -> '/content/gdrive/My Drive/Models/1597564312/variables'
'/content/pegasus/model/1597564312/variables/variables.data-00000-of-00001' -> '/content/gdrive/My Drive/Models/1597564312/variables/variables.data-00000-of-00001'
'/content/pegasus/model/1597564312/variables/variables.index' -> '/content/gdrive/My Drive/Models/1597564312/variables/variables.index'
'/content/pegasus/model/1597564312/saved_model.pb' -> '/content/gdrive/My Drive/Models/1597564312/saved_model.pb'


In [23]:
raise Exception("Model copied to {}, Factory Reset Runtime to predict".format(dir))

Exception: ignored

# Use saved model to predict 
**NOTE: Need to factory reset runtime before commencing**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# MODEL: gigaword for gigaword
#        cnn_dailymail for CNN Dailymail

# Aeslc
# 1597317576 - Batch of 8
# 1597369292 - Batch of 1
# 1597378344 - Batch of 16

# Base (params aeslc)
# 1597539174 - 4,512,32 4.28GB
# 1597541945 - 8,1024,32 14.99GB T4

# Base (params CNN)
# 1597558764 - 8,1024,128 10.6GB K80

# Base (params xsum)
# 1597564312 - 4,1024,64 8.14GB K80
MODELNAME = 'xsum'

MODEL_DIR = '1597564312'
BATCH_SIZE = 4
INPUT_SHAPE = 1024

In [3]:
%cd /content/

/content


In [4]:
!git clone http://github.com/rsin46/pegasus-demo.git
!git clone http://github.com/google-research/pegasus.git

Cloning into 'pegasus-demo'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 23 (delta 3), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.
Cloning into 'pegasus'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 353.09 KiB | 499.00 KiB/s, done.
Resolving deltas: 100% (56/56), done.


In [5]:
!mkdir /content/pegasus-demo/model
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.7MB/s 


In [6]:
# if MODELNAME == 'gigaword':
#   !cp '/content/gdrive/My Drive/Models/gigaword.tar.xz' '/content/pegasus-demo/model/gigaword.tar.xz'
#   !tar -xvf '/content/pegasus-demo/model/gigaword.tar.xz' -C '/content/pegasus-demo/model/' 
# elif MODELNAME == 'cnn_dailymail':
#   !cp '/content/gdrive/My Drive/Models/cnn_dailymail.xz' '/content/pegasus-demo/model/cnn_dailymail.xz'
#   !tar -xvf '/content/pegasus-demo/model/cnn_dailymail.xz' -C '/content/pegasus-demo/model/'
# else:
source = '/content/gdrive/My\ Drive/Models/{}'.format(MODEL_DIR)
!rsync -rv $source '/content/pegasus-demo/model/'

sending incremental file list
1597564312/
1597564312/saved_model.pb
1597564312/variables/
1597564312/variables/variables.data-00000-of-00001
1597564312/variables/variables.index

sent 2,280,597,497 bytes  received 89 bytes  24,391,418.03 bytes/sec
total size is 2,280,040,551  speedup is 1.00


In [7]:
%cd /content/pegasus-demo
# Initialising variables

TEXT = 'thermo.txt'
OUT_TEXT = f'/content/gdrive/My Drive/Models/{TEXT}_{MODELNAME}_{MODEL_DIR}.txt'
print(OUT_TEXT)

def append(line):
    with open(OUT_TEXT, 'a', buffering=1) as redf:
        redf.write(line)

/content/pegasus-demo
/content/gdrive/My Drive/Models/thermo.txt_xsum_1597564312.txt


In [ ]:
import text_eval
import public_parsing_ops
import tensorflow as tf
import numpy as np
import datetime
import time

_SPM_VOCAB = 'ckpt/c4.unigram.newline.10pct.96000.model'
encoder = public_parsing_ops.create_text_encoder("sentencepiece",
                                                 _SPM_VOCAB)

# shapes = {
#     'cnn_dailymail': (1024, 128),
#     'gigaword':(128, 32),
#     'aeslc':(512, 32),
#     'arxiv':(512, 64)
# }

# model_dir = {
#     'cnn_dailymail': 'model/1595037231/',
#     'gigaword': 'model/1595095633/',
#     'aeslc': 'model/{}'.format(MODEL_DIR),
#     'arxiv': 'model/{}'.format(MODEL_DIR)
# }

model_dir = 'model/{}'.format(MODEL_DIR)

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


imported = tf.saved_model.load(model_dir, tags='serve')


text = open(TEXT, "r", encoding="utf-8").read()

shape = INPUT_SHAPE
# shape,_ = shapes[MODELNAME]

input_ids_raw = encoder.encode(text)
input_ids_split = list(zip(*([iter(input_ids_raw)]*shape)))
batch_list = list(chunks(input_ids_split,BATCH_SIZE))

out_file = open(OUT_TEXT,"a")
current_time = datetime.datetime.now()
append(current_time.strftime("%d-%b-%Y (%H:%M:%S.%f)") + '\n')

for batch in batch_list:
  start = time.time()
  
  input_list = []

  for input_ids in batch:
    inputs = np.zeros(shape)
    idx = len(input_ids)
    if idx>shape: idx =shape

    inputs[:idx] = input_ids[:idx]

    example = tf.train.Example()

    example.features.feature["inputs"].int64_list.value.extend(inputs.astype(int))

    input_list.append(tf.constant([example.SerializeToString()]))
  
  # Pad if the last list doesn't fit into a batch
  if len(input_list) < BATCH_SIZE:
    for i in range(0,BATCH_SIZE - len(input_list)):
      input_list.append(input_list[-1])

  output = imported.signatures['serving_default'](examples=tf.concat(input_list, axis=0))
  prediction = text_eval.ids2str(encoder, output['outputs'].numpy(), None)
  end = time.time()
  duration = end - start
  print("{}\nPREDICTION >> ".format(duration), prediction)
  append(f'{duration}s ... {prediction} \n')


62.27459788322449
PREDICTION >>  Challenges to the Second Law of Thermodynamics Fundamental Theories of Physics An International Book Series on The Fundamental Theories of Physics: Their Clarification, Development and Application Editor: ALWYN VAN DER MERWE, University of Denver, U.S.A. HANS-J ⁇ RGEN TREDER, Zentralins 48 References 51 3 Modern Quantum Challenges: Theory 3.1 Prolegomenon . . . . . . . . . . . . . . . . . . . . . . . . 106 3.8.1 TC-GME and Mori . 107 3.8.2 TCL-GME and Tokuyama-Mori . 107 3.8.2 TC-GME and Tokuyama-Mori . 107 3.8.2 TC-GME and Tokuyama-Mori . 107 3.8.2 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
60.721498250961304
PREDICTION >>  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .175 6.2 Asymmetric Gravitator Model . .175 6.2 Asymmetric Gravitator Model . .175 6.2 Asymmetric Gravitator Model . .175 6.2 Asymmetric Gravitator Model . .175 6.2 Asymmetric Gravitator Model . .175 6.2 197 6.2.6 Laboratory Experiments . 202 6.3.1 Gr  

In [ ]:
infer = imported.signatures["serving_default"]
print(infer.inputs)
print(infer.output_shapes)

# GPU type

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("GPU type: " + gpu.name)
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

# Appendix

### Fine tuned model (change finetuned model name)

In [ ]:
!mkdir ckpt
!gsutil cp -r gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model
!gsutil -m cp -r gs://pegasus_ckpt/arxiv ckpt/pegasus_ckpt/

In [ ]:
!python3 pegasus/bin/train.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/arxiv/model.ckpt-340000 \
--model_dir=ckpt/pegasus_ckpt/arxiv --save_checkpoints_steps=100

In [ ]:
!cp -rv ckpt/pegasus_ckpt/arxiv/ /content/gdrive/My\ Drive/

## Analyse graph

In [ ]:
output = infer(tf.constant([example.SerializeToString()]))
text_eval.ids2str(encoder, output['outputs'].numpy(), None)

In [ ]:
input_serial = tf.concat([tf.constant([example.SerializeToString()]),tf.constant([example.SerializeToString()])],axis=0)

In [ ]:
input_serial

In [ ]:
%load_ext tensorboard

In [ ]:
dir = model_dir[MODELNAME]
!python import_pb_to_tensorboard.py --model_dir $dir --log_dir /tmp/tensorflow_logdir

In [ ]:
%tensorboard --logdir=/tmp/tensorflow_logdir